In [11]:
import requests
from requests import get
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import re

from time import sleep
from random import randint
import unicodedata

In [12]:
pages = np.arange(1, 501, 1)
pert=[1,3,4,6,7,8,9,11,13,14,15,16]

In [13]:
col=['nombre','fecha', 'periodo']
arreglo = np.arange(1, 32, 1)
for element in arreglo :
    col.append(element)
df=pd.DataFrame(columns=col)
df

,nombre,fecha,periodo,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31


In [14]:
for page in pages:
    for per in pert:
        response = requests.get("http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt="+str(page)+"&pert=" +str(per))
        html_doc = response.content
        soup = BeautifulSoup(html_doc, 'html.parser')
        sleep(randint(2,10))
        
        if per==1:
            periodo= "1_SO_1ano" 
        if per==3:
            periodo= "E_1R_1ano"
        if per==4:
            periodo= "2_SO_1ano"
        if per==6:
            periodo= "1_E_2R_1ano"
        if per==7:
            periodo= "2_E_2R_1ano"
        if per==8:
            periodo= "3_E_2R_1ano"
        if per==9:
            periodo= "1_SO_2ano"
        if per==11:
            periodo= "2_SO_2ano"
        if per==13:
            periodo= "1_E_2R_2ano"
        if per==14:
            periodo= "2_E_2R_2ano"
        if per==15:
            periodo= "3_E_2R_2ano"
        if per==16:
            periodo= "1_SO_3ano"
    
        cadena=soup.find_all('a', {'class': 'linkTitulo'})[0].get_text(strip=True)
        cadena= cadena.split(' ') 
        
        nombre= ""
        for p in cadena:
            if p!= '(DECESO)':
                nombre= nombre+p[0:].lower()
    
        nombre =nombre.split('(licencia)')
        nombre=nombre[0] 
        trans_tab = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
        nombre = unicodedata.normalize('NFKC', unicodedata.normalize('NFKD', nombre).translate(trans_tab))
        
        data = soup.find_all('td', {'width': ['49%', '13%']})
        tabla= []
        for i in range(1,len(data)):
            tabla.append(data[i].get_text(strip=True))
            
        corte=[]
        for t in tabla:
            if len(t)>7:
                c=tabla.index(t)
                corte.append(c)
                
        corte2=[]
        for t in tabla:
            if len(t)>7 and tabla.index(t)!=0:
                c=tabla.index(t)-1
                corte2.append(c)
        corte2.append(len(tabla)-1)
        
        lista_sup=[]

        for k,j in zip(corte, corte2):
            lista=[]
            lista.append(nombre)
            lista.append(tabla[k].lower())
            lista.append(periodo)
            for t in tabla[k+1:j+1]:
                if t!= '':
                    aux=re.split('(\d+)', t)
                    lista.append(aux[2])
            
            lista_sup.append(lista)
            i=len(df)+1
            df.loc[i]=lista
            
            
col_aux=[]
arreglo_aux = np.arange(1, 32, 1)
for element in arreglo_aux :
    col_aux.append(element)
    
df_aux = df.loc[0: ,col_aux]


df_count = (df_aux
      .stack()
      .groupby(level=0)
      .value_counts()
      .unstack(1)
      .add_prefix('count_')
     )

df_count = pd.concat([df, df_count], axis=1)

df_count.to_csv(r'C:\Users\Marlen\Documents\A.Marlen\A.datalab\asistencias.csv', index=False,header=True )

In [ ]:
df.tail(20)

In [17]:
df_count

,nombre,fecha,periodo,1,2,3,4,5,6,7,...,count_I,count_I/I,count_IJ,count_IJ/A,count_IJ/AO,count_IJ/IJ,count_IV,count_PM,count_PM/I,count_PM/PM
1,franciscojavierluevanonuñez,septiembre 2018,1_SO_1ano,A,,,A,,A,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,franciscojavierluevanonuñez,octubre 2018,1_SO_1ano,,A,A,A,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,franciscojavierluevanonuñez,noviembre 2018,1_SO_1ano,,,,,,A,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,franciscojavierluevanonuñez,diciembre 2018,1_SO_1ano,A,,,A,,A,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,franciscojavierluevanonuñez,enero 2019,E_1R_1ano,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12996,reginaldosandovalflores,julio 2020,3_E_2R_2ano,,,,,,,,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12997,reginaldosandovalflores,septiembre 2020,1_SO_3ano,A,A,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12998,reginaldosandovalflores,octubre 2020,1_SO_3ano,A,,,,,A,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12999,reginaldosandovalflores,noviembre 2020,1_SO_3ano,,,,A,A,,,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#col_aux=[]
#arreglo_aux = np.arange(1, 32, 1)
#for element in arreglo_aux :
#    col_aux.append(element)
    
#df_aux = df.loc[0:,col_aux]
#df_aux

In [19]:
#df_count = (df_aux
#      .stack()
#      .groupby(level=0)
#      .value_counts()
#      .unstack(1)
#     .add_prefix('count_')
#     )

#df_count = pd.concat([df, df_count], axis=1)
#df_count

,nombre,fecha,periodo,1,2,3,4,5,6,7,...,30,31,count_,count_A,count_A/A,count_A/I,count_AC,count_CS,count_IJ,count_PM
1,franciscojavierluevanonuñez,septiembre 2018,1_SO_1ano,A,,,A,,A,,...,,,21.0,9.0,NaN,NaN,NaN,NaN,NaN,1.0
2,franciscojavierluevanonuñez,octubre 2018,1_SO_1ano,,A,A,A,,,,...,A,A,20.0,9.0,NaN,NaN,2.0,NaN,NaN,NaN
3,franciscojavierluevanonuñez,noviembre 2018,1_SO_1ano,,,,,,A,,...,,,23.0,5.0,NaN,1.0,NaN,NaN,NaN,2.0
4,franciscojavierluevanonuñez,diciembre 2018,1_SO_1ano,A,,,A,,A,,...,,,21.0,9.0,NaN,NaN,1.0,NaN,NaN,NaN
5,franciscojavierluevanonuñez,enero 2019,E_1R_1ano,,,,,,,,...,,,30.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,franciscojavierluevanonuñez,febrero 2019,2_SO_1ano,A,,,,,A,A,...,,,22.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
7,franciscojavierluevanonuñez,marzo 2019,2_SO_1ano,,,,,IJ,,A,...,,,27.0,3.0,NaN,NaN,NaN,NaN,1.0,NaN
8,franciscojavierluevanonuñez,abril 2019,2_SO_1ano,,A,A,A,,,,...,A,,18.0,10.0,1.0,NaN,NaN,NaN,2.0,NaN
9,franciscojavierluevanonuñez,mayo 2019,1_E_2R_1ano,,,,,,,,...,,,30.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
10,franciscojavierluevanonuñez,mayo 2019,2_E_2R_1ano,,,,,,,,...,,,30.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [10]:
#df.to_csv(r'C:\Users\Marlen\Documents\A.Marlen\A.datalab\asistencias_prueba.csv', index=False,header=True )